In [1]:
from glob import glob
import librosa
import soundfile as sf
import numpy as np
import pandas as pd
import sys
import time
import datetime
from tqdm import tqdm
import random
import os
import gc
import cv2

from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, AdamW, lr_scheduler
from torch.distributions import Uniform
from torch.utils.data import DataLoader, Dataset

from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation

from efficientnet_pytorch import EfficientNet
from torchvision.models import resnet34, resnet50

import sys
sys.path.append('..')
from libs import transform as tr
from libs import spectrogram as spec
from libs import criterion as cr

import warnings
warnings.filterwarnings("ignore")

/home/yuigahama/anaconda3/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [2]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = False  # type: ignore
set_seed(53)

In [3]:
def normalize_melspec(X: np.ndarray):
    eps = 1e-6
    mean = X.mean()
    X = X - mean
    std = X.std()
    Xstd = X / (std + eps)
    norm_min, norm_max = Xstd.min(), Xstd.max()
    if (norm_max - norm_min) > eps:
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

In [4]:
train_tp = pd.read_csv('../../data/train_tp.csv')
train_fp = pd.read_csv('../../data/train_fp.csv')
submission = pd.read_csv('../../data/sample_submission.csv')

pred_target = list(submission.columns)[1:]

SR = 48000

In [5]:
class RfcxDataSet(Dataset):
    def __init__(self,
                 tp:pd.DataFrame,
                 fp:pd.DataFrame,
                 train: bool,
                 data_path:str,
                 pcen_parameters:dict,
                 pre_calc=True,
                 n_mels=128
    ):
        self.tp = tp
        self.fp = fp
        self.path = data_path
        self.img_size = 256
        self.train = train
        self.n_mels = n_mels
        self.pre_calc = pre_calc
        
        self.pcen_parameters = pcen_parameters
        
    def __len__(self):
        return len(self.tp)
    
    def load(self, record_path):
        y, orig_sr = sf.read(record_path)
        
        if orig_sr != SR:
            y = librosa.resample(y, orig_sr=orig_sr, target_sr=SR, res_type="kaiser_best")
        return y
    
    def get_duration(self, t_min, t_max, duration=10):
        annotated_duration = t_max - t_min
        
        if annotated_duration > duration:
            limit_sec = t_max - duration
            start_sec = random.randint(t_min, limit_sec)
            end_sec = start_sec + duration

        else:
            res_time = duration - annotated_duration
            front_limit = res_time if res_time < t_min else t_min
            
            front_time = random.randint(0, front_limit)
            
            back_limit = 60 - t_max
            
            tmp_time = res_time - front_time
            back_time = tmp_time if tmp_time < back_limit else back_limit
            
            if not tmp_time < back_limit:
                front_time += tmp_time - back_limit
            
            start_sec = t_min - front_time
            end_sec = t_max + back_time
            
        return start_sec, end_sec
    
    def create_mel(self, y, target_fp):
        melspec = librosa.feature.melspectrogram(
            y,
            sr=SR,
            fmin=0,
            fmax=15000,
            n_mels=128
        )

        f, m = melspec.shape
        time_rate = m / 60
        freq_rate = f / 15000

        for i in range(len(target_fp)):
            sample = target_fp.iloc[i]
            f_min = int(sample['f_min'] * freq_rate)
            f_max = int(sample['f_max'] * freq_rate)

            melspec[f_min:f_max, :] = 1e-4

        pcen = librosa.pcen(melspec, sr=SR, **self.pcen_parameters)
        clean_mel = librosa.power_to_db(melspec ** 1.5)
        melspec = librosa.power_to_db(melspec)

        norm_melspec = normalize_melspec(melspec)
        norm_pcen = normalize_melspec(pcen)
        norm_clean_mel = normalize_melspec(clean_mel)

        image = np.stack([norm_melspec, norm_pcen, norm_clean_mel], axis=-1)

        return image
    
    def __getitem__(self, idx: int):
        sample = self.tp.iloc[idx, :]
        recording_id = sample['recording_id']
        t_min = int(round(sample['t_min']))
        t_max = int(round(sample['t_max']))
        
        target_fp = self.fp.query(f'recording_id == "{recording_id}"')
        
        if self.pre_calc:
            image = torch.load(self.path + recording_id + '.tensor')
        else:
            record_path = self.path + recording_id + '.flac'
            y = self.load(record_path)
            image = self.create_mel(y, target_fp)
        
        time_rate = int(image.shape[1] / 60)
        
        start_sec, end_sec = self.get_duration(t_min, t_max)
        
        start = int(start_sec * time_rate)
        end = int(end_sec * time_rate)
        
        image = image[:, start:end, :]
        
        height, width, _ = image.shape
        image = cv2.resize(image, (self.img_size * 2, self.img_size))
        image = np.moveaxis(image, 2, 0)
        image = (image / 255.0).astype(np.float32)
        
        species_id = sample['species_id']
        target = torch.zeros([24], dtype=torch.float32)
        target[species_id] = 1
        
        return image, target

In [6]:
class RfcxVal(Dataset):
    def __init__(self,
                 tp:pd.DataFrame,
                 train: bool,
                 data_path:str,
                 pcen_parameters:dict,
                 pre_calc=True,
                 n_mels=128
    ):
        self.tp = tp
        self.path = data_path
        self.img_size = 256
        self.train = train
        self.n_mels = n_mels
        self.pre_calc = pre_calc
        
        self.pcen_parameters = pcen_parameters
        
    def __len__(self):
        return len(self.tp)
    
    def load(self, record_path):
        y, orig_sr = sf.read(record_path)
        
        if orig_sr != SR:
            y = librosa.resample(y, orig_sr=orig_sr, target_sr=SR, res_type="kaiser_best")
        return y
    
    def create_mel(self, y):
        melspec = librosa.feature.melspectrogram(
            y,
            sr=SR,
            fmin=0,
            fmax=15000,
            n_mels=128
        )
        pcen = librosa.pcen(melspec, sr=SR, **self.pcen_parameters)
        clean_mel = librosa.power_to_db(melspec ** 1.5)
        melspec = librosa.power_to_db(melspec)

        norm_melspec = normalize_melspec(melspec)
        norm_pcen = normalize_melspec(pcen)
        norm_clean_mel = normalize_melspec(clean_mel)

        image = np.stack([norm_melspec, norm_pcen, norm_clean_mel], axis=-1)

        return image
    
    def __getitem__(self, idx: int):
        sample = self.tp.iloc[idx, :]
        recording_id = sample['recording_id']
        
        if self.pre_calc:
            image_ = torch.load(self.path + recording_id + '.tensor')
        else:
            record_path = self.path + recording_id + '.flac'
            y = self.load(record_path)
            image_ = self.create_mel(y, target_fp)
        
        time_rate = int(image_.shape[1] / 60)
        
        images = []
        
        for s in range(0,60,10):
            start_sec = s
            end_sec   = s + 10

            start = int(start_sec * time_rate)
            end = int(end_sec * time_rate)
        
            image = image_[:, start:end, :]
                        
            height, width, _ = image.shape
            image = cv2.resize(image, (self.img_size * 2, self.img_size))
            image = np.moveaxis(image, 2, 0)
            image = (image / 255.0).astype(np.float32)
            
            images.append(image)
        
        species_id = sample['species_id']
        target = torch.zeros([24], dtype=torch.float32)
        target[species_id] = 1
        
        return images, target

In [7]:
class SimpleRfcx(Dataset):
    def __init__(self, df, path, train=True):
        self.df = df
        self.path = path
        self.train = train
        self.img_size = 256
        
    def __len__(self):
        return len(self.df)
    
    def get_duration(self, t_min, t_max, duration=10):
        annotated_duration = t_max - t_min
        
        if annotated_duration > duration:
            limit_sec = t_max - duration
            start_sec = random.randint(t_min, limit_sec)
            end_sec = start_sec + duration
            
            start = start_sec * SR
            end = end_sec * SR
        else:
            res_time = duration - annotated_duration
            front_limit = res_time if res_time < t_min else t_min
            
            front_time = random.randint(0, front_limit)
            
            back_limit = 60 - t_max
            
            tmp_time = res_time - front_time
            back_time = tmp_time if tmp_time < back_limit else back_limit
            
            if not tmp_time < back_limit:
                front_time += tmp_time - back_limit
            
            start = (t_min - front_time) * SR
            end = (t_max + back_time) * SR
            
        return start, end
    
        
    def __getitem__(self, idx: int):
        sample = self.df.iloc[idx, :]
        recording_id = sample['recording_id']
        data = torch.load(
            self.path + recording_id + '.tensor'
        )
        
        image_ = data
            
        time_rate = int(image_.shape[1] / 60)
        
        images = []
        
        for s in range(0,60,10):
            start_sec = s
            end_sec   = s + 10

            start = int(start_sec * time_rate)
            end = int(end_sec * time_rate)
        
            image = image_[:, start:end, :]
            
            height, width, _ = image.shape
            image = cv2.resize(image, (self.img_size * 2, self.img_size))
            image = np.moveaxis(image, 2, 0)
            image = (image / 255.0).astype(np.float32)
            
            images.append(image)
        
        return images

In [8]:
class BaseModel(nn.Module):
    def __init__(self, model_type, model_name, output_size):
        super().__init__()
        self.model_type = model_type
        
        self.mixup_alpha = 0.2
        self.random_state = np.random.RandomState(123)
        
        base_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
        layers = list(base_model.children())[:-1]
        self.encoder = nn.Sequential(*layers)
        in_features = base_model.fc.in_features
        self.fc1 = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(1024, output_size)
        )
        
    def forward(self, x):
        """x = x.transpose(1, 2)
        x = self.bn0(x)
        x = x.transpose(1, 2)"""
        
        x = self.encoder(x)
            
        x = self.fc1(x)
        return x

In [9]:
def save(fold, model, optim, criterion, file_path="../../model/"):
    if not TEST_NAME in os.listdir(file_path):
        os.mkdir(file_path+TEST_NAME)
    
    
    output_path = file_path + TEST_NAME + '/' + f"{TEST_NAME}_{fold}.model"
    
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.cpu().state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'criterion': criterion
        },
        output_path)
    
    model.to(device)
    
    return output_path

In [10]:
# LRAP. Instance-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = (scores.sum(-1) / labels.sum(-1)).mean()
    return score.item()

# label-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LWLRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    # Number of GT labels per instance
    num_labels = labels.sum(-1)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = scores.sum() / labels.sum()
    return score.item()

def mixup_socre(cor, x, y, mix_info):
    return cor(x, y) * mix_info['lam'] + cor(x, y[mix_info['index']].squeeze()) * (1-mix_info['lam'])

# Sample usage
# y_true = torch.tensor(np.array([[1, 1, 0], [1, 0, 1], [0, 0, 1]]))
# y_score = torch.tensor(np.random.randn(3, 3))
# print(LRAP(y_score, y_true), LWLRAP(y_score, y_true))

In [11]:
TEST_NAME = 'baseline-pic-res'

n_splits = 5
random_state = 1
epochs = 75

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_params = {
    'model_type': 'res', # res or efficientnet
    'model_name': 'resnet50',
    'output_size': 24,
}

optim_params = {
    'lr': 1e-3,
    'weight_decay': 5e-5,
    'betas': (0.9, 0.999)
}

scheduler_params = {
    'mode': 'max',
    'patience': 1,
    'factor': 0.6,
    'verbose': False
}

pcen_parameters = {
    'gain': 0.98,
    'bias': 2,
    'power': 0.5,
    'time_constant': 0.4,
    'eps': 0.000001,
}

train_params = {
    'fp': train_fp,
    'pcen_parameters': pcen_parameters,
    'train': True,
    'data_path': '/home/yuigahama/kaggle/rfcx/data/train_noise_v2/'  
}

val_params = {
    'fp': train_fp,
    'train': True,
    'pcen_parameters': pcen_parameters,
    'data_path': '/home/yuigahama/kaggle/rfcx/data/train_wo_fp/'
}

test_data_params = {
    'train': False,
    'path': '/home/yuigahama/kaggle/rfcx/data/test_wo_fp/'
}

dataloder_params = {
    'batch_size': 16,
    'num_workers': 15,
    'pin_memory': False,
}

test_dataloder_params = {
    'batch_size': 32,
    'num_workers': 15,
    'pin_memory': False,
    'shuffle':False
}

In [ ]:
tta = np.zeros((len(submission), 24))
cv_score = 0

for fold_id, (train_index, val_index) in enumerate(skf.split(train_tp, train_tp.species_id)):
    print(f'---------- fold {fold_id} ----------')
    
    model = BaseModel(**model_params).to(device)
    optim = Adam(model.parameters(), **optim_params)
    #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer=optim, **scheduler_params)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer=optim, T_max=10)
    
    pos_weights = torch.ones(24)
    pos_weights = pos_weights * 24
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
    
    train_dataset = RfcxDataSet(train_tp.iloc[train_index], **train_params)
    val_dataset   = RfcxDataSet(train_tp.iloc[val_index], **val_params)
    test_dataset  = SimpleRfcx(submission, **test_data_params)

    train_dataloader = DataLoader(train_dataset, shuffle=True, **dataloder_params)
    val_dataloader = DataLoader(val_dataset, shuffle=False, **dataloder_params)
    test_dataloader = DataLoader(test_dataset, **test_dataloder_params)
    
    es = 15
    for epoch in range(1, epochs):
        if es == 0:
            break
        
        start_time = time.time()
        bast_score = 0
        
        # train
        model.train()
        train_loss = 0
        train_score = 0
        train_corr = 0
        
        for data in train_dataloader:
            image = data[0].float().to(device)
            label = data[1]
            
            optim.zero_grad()
            output = model(image)
            

            loss = criterion(output.cpu(), label)
            score = LWLRAP(torch.sigmoid(output).cpu(), label)
            
            loss.backward()
            optim.step()
            
            vals, answers = torch.max(output, 1)
            vals, targets = torch.max(label, 1)
            corrects = 0
            for i in range(0, len(answers)):
                if answers[i] == targets[i]:
                    corrects = corrects + 1
                    
            
            train_corr += corrects
            train_loss += loss.item()
            train_score += score
            
        train_loss  /= len(train_dataloader)
        train_score /= len(train_dataloader)
        
        # val
        model.eval()
        val_loss = 0
        val_score = 0
        val_corr = 0
        
        with torch.no_grad():
            for val_data in val_dataloader:
                '''target = val_data[1]
                num = len(val_data[0])
                label = val_data[1]
                tmp = torch.zeros((val_data[0][0].size(0), 24))
                
                for img in val_data[0]:
                    output = model(img.to(device))
                    tmp += output.cpu() / num'''
                
                image = val_data[0].float().to(device)
                label = val_data[1]

                output = model(image)
                    
                vals, answers = torch.max(output, 1)
                vals, targets = torch.max(label, 1)
                corrects = 0
                for i in range(0, len(answers)):
                    if answers[i] == targets[i]:
                        corrects = corrects + 1

                val_corr += corrects
                val_loss += criterion(output.cpu(), label).item()
                val_score += LWLRAP(torch.sigmoid(output).cpu(), label)
                
        val_loss  /= len(val_dataloader)
        val_score /= len(val_dataloader)
        
        duration = str(datetime.timedelta(seconds=time.time() - start_time))[:7]
        print(f'epoch {epoch:3}| T | L: {train_loss:.3} | S: {train_score:.3} | C: {train_corr}/{len(train_dataset)} | V | loss: {val_loss:.3} | score: {val_score:.3} | C: {val_corr}/{len(val_dataset)} | time: {duration}')

        if bast_score < val_score:
            bast_score = val_score
            bast_path = save(fold_id, model, optim, criterion)
        else:
            es -= 1
        
        scheduler.step(val_score)
            
        
    oof = np.zeros((len(test_dataset), 24))
    model = BaseModel(**model_params).to(device)
    
    bast_model_parms = torch.load(bast_path)
    model.load_state_dict(bast_model_parms['model_state_dict'])

    
    with torch.no_grad():
        start = 0
        for i, test_data in enumerate(test_dataloader):
            start = 0
            target_size = test_data[0].size(0)
            for d in test_data:
                image = d.float().to(device)
                end = start + target_size
            
                output = model(image)

            output = torch.sigmoid(output)
            output = output.detach().cpu().numpy()
            
            oof[start:end, :] = output
            start = end
            
    tta += (oof / n_splits)
    
    del model, train_dataset, val_dataset, test_dataset, train_dataloader, val_dataloader, test_dataloader
    del bast_model_parms, optim
    gc.collect()

Using cache found in /home/yuigahama/.cache/torch/hub/zhanghang1989_ResNeSt_master


---------- fold 0 ----------
epoch   1| T | L: 1.33 | S: 0.213 | C: 85/972 | V | loss: 1.26 | score: 0.268 | C: 23/244 | time: 0:00:20
epoch   2| T | L: 1.15 | S: 0.267 | C: 95/972 | V | loss: 1.14 | score: 0.272 | C: 21/244 | time: 0:00:19
epoch   3| T | L: 1.11 | S: 0.284 | C: 101/972 | V | loss: 1.13 | score: 0.305 | C: 30/244 | time: 0:00:19
epoch   4| T | L: 1.07 | S: 0.296 | C: 110/972 | V | loss: 1.11 | score: 0.3 | C: 31/244 | time: 0:00:19
epoch   5| T | L: 1.03 | S: 0.322 | C: 137/972 | V | loss: 1.27 | score: 0.29 | C: 24/244 | time: 0:00:19
epoch   6| T | L: 1.02 | S: 0.348 | C: 160/972 | V | loss: 1.07 | score: 0.337 | C: 39/244 | time: 0:00:19
epoch   7| T | L: 1.02 | S: 0.36 | C: 173/972 | V | loss: 1.03 | score: 0.339 | C: 38/244 | time: 0:00:19
epoch   8| T | L: 0.946 | S: 0.385 | C: 191/972 | V | loss: 1.04 | score: 0.355 | C: 47/244 | time: 0:00:19
epoch   9| T | L: 0.929 | S: 0.393 | C: 192/972 | V | loss: 1.2 | score: 0.385 | C: 50/244 | time: 0:00:19
epoch  10| T 

Using cache found in /home/yuigahama/.cache/torch/hub/zhanghang1989_ResNeSt_master


---------- fold 1 ----------


Using cache found in /home/yuigahama/.cache/torch/hub/zhanghang1989_ResNeSt_master


...

epoch   1| T | L: 1.31 | S: 0.224 | C: 73/973 | V | loss: 1.56 | score: 0.238 | C: 14/243 | time: 0:00:20
epoch   2| T | L: 1.16 | S: 0.273 | C: 97/973 | V | loss: 1.2 | score: 0.255 | C: 22/243 | time: 0:00:20
epoch   3| T | L: 1.11 | S: 0.283 | C: 110/973 | V | loss: 1.14 | score: 0.247 | C: 19/243 | time: 0:00:20
epoch   4| T | L: 1.07 | S: 0.295 | C: 110/973 | V | loss: 1.16 | score: 0.252 | C: 22/243 | time: 0:00:20
epoch   5| T | L: 1.06 | S: 0.312 | C: 127/973 | V | loss: 1.18 | score: 0.306 | C: 28/243 | time: 0:00:20
epoch   6| T | L: 1.02 | S: 0.305 | C: 108/973 | V | loss: 1.09 | score: 0.306 | C: 33/243 | time: 0:00:20
epoch   7| T | L: 1.01 | S: 0.326 | C: 130/973 | V | loss: 1.21 | score: 0.29 | C: 28/243 | time: 0:00:20
epoch   8| T | L: 1.0 | S: 0.322 | C: 128/973 | V | loss: 1.26 | score: 0.241 | C: 16/243 | time: 0:00:20
epoch   9| T | L: 1.03 | S: 0.32 | C: 127/973 | V | loss: 1.23 | score: 0.317 | C: 32/243 | time: 0:00:20
epoch  10| T | L: 1.01 | S: 0.348 | C: 159/

Using cache found in /home/yuigahama/.cache/torch/hub/zhanghang1989_ResNeSt_master


---------- fold 3 ----------


Using cache found in /home/yuigahama/.cache/torch/hub/zhanghang1989_ResNeSt_master


epoch   1| T | L: 1.35 | S: 0.204 | C: 75/973 | V | loss: 1.35 | score: 0.231 | C: 22/243 | time: 0:00:20
epoch   2| T | L: 1.35 | S: 0.207 | C: 72/973 | V | loss: 1.44 | score: 0.217 | C: 20/243 | time: 0:00:20
epoch   3| T | L: 1.34 | S: 0.206 | C: 75/973 | V | loss: 1.34 | score: 0.219 | C: 20/243 | time: 0:00:20
epoch   4| T | L: 1.31 | S: 0.215 | C: 77/973 | V | loss: 1.33 | score: 0.236 | C: 20/243 | time: 0:00:20
epoch   5| T | L: 1.29 | S: 0.226 | C: 70/973 | V | loss: 1.29 | score: 0.213 | C: 15/243 | time: 0:00:20
epoch   6| T | L: 1.28 | S: 0.243 | C: 88/973 | V | loss: 1.28 | score: 0.234 | C: 18/243 | time: 0:00:20
epoch   7| T | L: 1.25 | S: 0.262 | C: 95/973 | V | loss: 1.26 | score: 0.248 | C: 19/243 | time: 0:00:20
epoch   8| T | L: 1.19 | S: 0.266 | C: 89/973 | V | loss: 1.16 | score: 0.286 | C: 24/243 | time: 0:00:20
epoch   9| T | L: 1.13 | S: 0.281 | C: 97/973 | V | loss: 1.13 | score: 0.276 | C: 22/243 | time: 0:00:20
epoch  10| T | L: 1.14 | S: 0.278 | C: 91/973 

In [16]:
print(cv_score)

0


In [17]:
pred_sub = pd.DataFrame(tta, columns=pred_target)
pred_sub

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
0,0.445695,0.533450,0.004240,0.403188,0.435533,0.120745,0.001689,0.015439,0.575393,0.043221,...,0.144612,0.057418,0.056819,0.007540,0.602144,0.004007,0.003464,0.398680,0.111421,0.131720
1,0.201426,0.044273,0.329872,0.372739,0.002411,0.564328,0.394738,0.043304,0.004310,0.162767,...,0.140225,0.125184,0.034053,0.237484,0.086417,0.380273,0.151399,0.001576,0.406527,0.478254
2,0.017627,0.036188,0.091064,0.425175,0.031050,0.193798,0.468982,0.373206,0.010106,0.503285,...,0.265688,0.327697,0.184743,0.670946,0.009635,0.224816,0.405624,0.001498,0.187714,0.367208
3,0.382605,0.257048,0.142256,0.518494,0.125351,0.665855,0.113524,0.229342,0.275768,0.223545,...,0.278501,0.200804,0.145173,0.171095,0.205457,0.136477,0.094496,0.088500,0.296206,0.394871
4,0.035332,0.024020,0.074555,0.436384,0.028268,0.327446,0.317773,0.703383,0.011371,0.458428,...,0.224739,0.362348,0.140191,0.498871,0.017267,0.212362,0.254635,0.007153,0.426913,0.577057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1988,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1990,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [18]:
pd.concat([submission['recording_id'], pred_sub], axis=1).to_csv('submission.csv', index=False)